In [6]:
# Requires GenePattern Notebook: pip install genepattern-notebook
import gp
import genepattern

# Username and password removed for security reasons.
genepattern.display(genepattern.session.register("https://cloud.genepattern.org/gp", "", ""))

GPAuthWidget()

- https://datasets.genepattern.org/data/module_support_files/Conos/data/conos_data/pbmc.txt
- https://datasets.genepattern.org/data/module_support_files/Conos/HNSCC_noribo.txt
- https://datasets.genepattern.org/data/module_support_files/Conos/MEL_noribo.txt
- https://datasets.genepattern.org/data/module_support_files/Conos/CRC_noribo.txt

In [7]:
conos_preprocess_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00409')
conos_preprocess_job_spec = conos_preprocess_task.make_job_spec()
conos_preprocess_job_spec.set_parameter("file_list", ["https://datasets.genepattern.org/data/module_support_files/Conos/data/conos_data/pbmc.txt", "https://datasets.genepattern.org/data/module_support_files/Conos/HNSCC_noribo.txt", "https://datasets.genepattern.org/data/module_support_files/Conos/MEL_noribo.txt"])
conos_preprocess_job_spec.set_parameter("knn", "40")
conos_preprocess_job_spec.set_parameter("perplexity", "50")
conos_preprocess_job_spec.set_parameter("pagoda_odgenes", "3000")
conos_preprocess_job_spec.set_parameter("projection_method", "PCA")
conos_preprocess_job_spec.set_parameter("ncomps", "50")
conos_preprocess_job_spec.set_parameter("conos_odgenes", "1000")
conos_preprocess_job_spec.set_parameter("job.memory", "4Gb")
conos_preprocess_job_spec.set_parameter("job.queue", "gp-cloud-default")
conos_preprocess_job_spec.set_parameter("job.cpuCount", "1")
conos_preprocess_job_spec.set_parameter("job.walltime", "02:00:00")
genepattern.display(conos_preprocess_task)

job197962 = gp.GPJob(genepattern.session.get(0), 197962)
genepattern.display(job197962)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00409')

GPJobWidget(job_number=197962)

In [8]:
conos_cluster_task = gp.GPTask(genepattern.session.get(0), 'urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00410')
conos_cluster_job_spec = conos_cluster_task.make_job_spec()
conos_cluster_job_spec.set_parameter("conos_object", "https://cloud.genepattern.org/gp/jobResults/197928/conos_preprocess_output.rds")
conos_cluster_job_spec.set_parameter("runleiden", "True")
conos_cluster_job_spec.set_parameter("leiden_resolution", "1.0")
conos_cluster_job_spec.set_parameter("runwalktrap", "True")
conos_cluster_job_spec.set_parameter("walktrap_steps", "10")
conos_cluster_job_spec.set_parameter("umap_distance", "0.05")
conos_cluster_job_spec.set_parameter("umap_spread", "5")
conos_cluster_job_spec.set_parameter("job.memory", "4Gb")
conos_cluster_job_spec.set_parameter("job.queue", "gp-cloud-default")
conos_cluster_job_spec.set_parameter("job.cpuCount", "3")
conos_cluster_job_spec.set_parameter("job.walltime", "02:00:00")
genepattern.display(conos_cluster_task)


job197957 = gp.GPJob(genepattern.session.get(0), 197957)
genepattern.display(job197957)

GPTaskWidget(lsid='urn:lsid:broad.mit.edu:cancer.software.genepattern.module.analysis:00410')

GPJobWidget(job_number=197957)

In [4]:
%load_ext rpy2.ipython
%load_ext nbtools.r_support
import genepattern
@genepattern.build_ui(name="Import Conos.Cluster output for further analysis", parameters={
    "conosgraph": {
        "name": "Conos Cluster Output:",
        "description": "conos_cluster_output.rds from Conos.Cluster module",
        "type": "file",
        "kinds": ["rds"]
    },
    "output_var": {
    "name": "results",
    "description": "",
    "default": "quantification_source",
    "hide": True
    }
})

def get_files_function(conosgraph):
#     print(conosgraph)

    f = gp.GPFile(genepattern.session.get(0), conosgraph)
    open("conos_cluster_output.rds", 'wb').write(f.open().read())
    print('done!')

newrds = "unread"
%Rpush newrds

UIBuilder(function_import='nbtools.tool(id="Import Conos.Cluster output for further analysis", origin="Noteboo…

In [13]:
%%r_build_ui {"name": "Plot gene expression on the joint graph", "parameters": {"geneofinterest": { "name": "Gene Identifier:", "description": "A gene identifier to plot expression on the embedded map", "type": "text"}, "paintcluster": { "name": "Show Clustering:", "description": "Option to show clustering information beside gene expression", "type": "choice", "choices": {"None": "none", "Show Leiden": "leiden", "Show Walktrap": "walktrap", "Show Both Leiden and Walktrap": "both"}, "default": "none"},  "output_var": { "hide": "True" } }}

customplot <- function(geneofinterest,paintcluster){
    if (newrds == "unread") {
    conos <- readRDS("conos_cluster_output.rds")    
    con <- conos$con
    runleiden <- conos$runleiden
    runwalktrap <- conos$runwalktrap
    newrds <- "read"
    }

    if (paintcluster == "none"){
    print(con$plotPanel(alpha=0.1, color.by='gene', gene=paste0(geneofinterest)))}

    if (runleiden == TRUE & paintcluster == "leiden"){
#     pdf(paste0("Leiden_Clusters_with_",geneofinterest,"_Expression.pdf"), width=14.88, height=8.2)
#     print(cowplot::plot_grid(con$plotPanel(alpha=0.1, clustering='leiden'),
#     con$plotPanel(alpha=0.1, color.by='gene', title=paste0(geneofinterest,"_Expression"), gene=paste0(geneofinterest))))
#     dev.off()
    print(cowplot::plot_grid(con$plotPanel(alpha=0.1, clustering='leiden'),
    con$plotPanel(alpha=0.1, color.by='gene', gene=paste0(geneofinterest))))}

    if (runwalktrap == TRUE & paintcluster == "walktrap"){
#     pdf(paste0("Walktrap_Clusters_with_",geneofinterest,"_Expression.pdf"), width=14.88, height=8.2)
#     print(cowplot::plot_grid(con$plotPanel(alpha=0.1, clustering='walktrap'),
#     con$plotPanel(alpha=0.1, color.by='gene', title=paste0(geneofinterest,"_Expression"), gene=paste0(geneofinterest))))
#     dev.off()
    print(cowplot::plot_grid(con$plotPanel(alpha=0.1, clustering='walktrap'),
    con$plotPanel(alpha=0.1, color.by='gene', gene=paste0(geneofinterest))))}

    if (runleiden == TRUE & runwalktrap == TRUE & paintcluster == "both"){
#     pdf(paste0("Leiden_and_Walktrap_Clusters_with_",geneofinterest,"_Expression.pdf"), width=14.88, height=8.2)
#     print(cowplot::plot_grid(con$plotPanel(alpha=0.1, clustering='leiden'), con$plotPanel(alpha=0.1, clustering='walktrap'),
#     con$plotPanel(alpha=0.1, color.by='gene', title=paste0(geneofinterest,"_Expression"), gene=paste0(geneofinterest))))
#     dev.off()
    print(cowplot::plot_grid(con$plotPanel(alpha=0.1, clustering='leiden'), con$plotPanel(alpha=0.1, clustering='walktrap'),
    con$plotPanel(alpha=0.1, color.by='gene', gene=paste0(geneofinterest))))}
    return("")
}
customplot(geneofinterest,paintcluster)

UIBuilder(function_import='nbtools._r_wrappers["E64BC3B0146AE883BB7EEBFEE7B2DF92"]', name='Plot gene expressio…